In [1]:
# Project Title
# Downloading Dataset
# Data Preparation and Cleaning
# Exploratory Analysis and visualization
# Question and Answer
# Infrences and Conclusion
# References and Future work

# Project Title

IPL Dataset analaysis

# Downloading Dataset

# Random

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
match_df = pd.read_csv('IPL_dataset/matches.csv')
deliveries_df = pd.read_csv('IPL_dataset/deliveries.csv')
ras_df = pd.read_csv('IPL_dataset/most_runs_average_strikerate.csv')

In [4]:
innings1_df = deliveries_df[deliveries_df['inning']==1] # First Innings data
innings2_df = deliveries_df[deliveries_df['inning']==2] # Second Innings data

## Boundary(4&6) Hitting Record for MVP batsman

### V Kohli : 1st Innings

In [5]:
kohli6a_df = innings1_df.loc[(innings1_df['batsman']=='V Kohli') &(innings1_df['batsman_runs']==6)] # 6 data for V Kohli
kohli6a_main_df = kohli6a_df[['bowling_team', 'over', 'ball', 'bowler']] # Filtered 6 data for Kohli
kohli6a_over_df = kohli6a_main_df[['over']].value_counts() # number of 6's hit per over by Kohli

In [6]:
kohli4a_df = innings1_df.loc[(innings1_df['batsman']=='V Kohli') &(innings1_df['batsman_runs']==4)] # 4 data for V Kohli
kohli4a_main_df = kohli4a_df[['bowling_team', 'over', 'ball', 'bowler']] # Filtered 4 data for Kohli
kohli4a_over_df = kohli4a_main_df[['over']].value_counts() # number of 4's hit per over by Kohli

### V Kohli: 2nd Innings

In [7]:
# Record for 6 by Kohli

kohli6b_df = innings2_df.loc[(innings2_df['batsman']=='V Kohli') &(innings2_df['batsman_runs']==6)] # 6 data for V Kohli
kohli6b_main_df = kohli6b_df[['bowling_team', 'over', 'ball', 'bowler']] # Filtered 6 data for Kohli
kohli6b_over_df = kohli6b_main_df[['over']].value_counts() # number of 6's hit per over by Kohli

In [8]:
#  Record for 4 by Kohli

kohli4b_df = innings2_df.loc[(innings2_df['batsman']=='V Kohli') &(innings2_df['batsman_runs']==4)] # 4 data for V Kohli
kohli4b_main_df = kohli4b_df[['bowling_team', 'over', 'ball', 'bowler']] # Filtered 4 data for Kohli
kohli4b_over_df = kohli4b_main_df[['over']].value_counts() # number of 4's hit per over by Kohli

### S Raina: 1st Innings

In [9]:
raina6a_df = innings1_df.loc[(innings1_df['batsman']=='SK Raina') &(innings1_df['batsman_runs']==6)] # 6 data for S Raina
raina6a_main_df = raina6a_df[['bowling_team', 'over', 'ball', 'bowler']] # Filtered 6 data for Raina
raina6a_over_df = raina6a_main_df[['over']].value_counts() # number of 6's hit per over by Raina

In [10]:
raina4a_df = innings1_df.loc[(innings1_df['batsman']=='SK Raina') &(innings1_df['batsman_runs']==4)] # 4 data for S Raina
raina4a_main_df = raina4a_df[['bowling_team', 'over', 'ball', 'bowler']] # Filtered 4 data for Raina
raina4a_over_df = raina4a_main_df[['over']].value_counts() # number of 4's hit per over by Raina

### S Raina: 2nd Innings

In [11]:
raina6b_df = innings2_df.loc[(innings2_df['batsman']=='SK Raina') &(innings2_df['batsman_runs']==6)] # 6 data for S Raina
raina6b_main_df = raina6b_df[['bowling_team', 'over', 'ball', 'bowler']] # Filtered 6 data for Raina
raina6b_over_df = raina6b_main_df[['over']].value_counts() # number of 6's hit per over by Raina

In [12]:
raina4b_df = innings2_df.loc[(innings2_df['batsman']=='SK Raina') &(innings2_df['batsman_runs']==4)] # 4 data for S Raina
raina4b_main_df = raina4b_df[['bowling_team', 'over', 'ball', 'bowler']] # Filtered 4 data for Raina
raina4b_over_df = raina4b_main_df[['over']].value_counts() # number of 4's hit per over by Raina

### R Sharma: 1st Innings

In [13]:
sharma6a_df = innings1_df.loc[(innings1_df['batsman']=='RG Sharma') &(innings1_df['batsman_runs']==6)] # 6 data for R Sharma
sharma6a_main_df = sharma6a_df[['bowling_team', 'over', 'ball', 'bowler']] # Filtered 6 data for Sharma
sharma6a_over_df = sharma6a_main_df[['over']].value_counts() # number of 6's hit per over by Sharma

In [14]:
sharma4a_df = innings1_df.loc[(innings1_df['batsman']=='RG Sharma') &(innings1_df['batsman_runs']==4)] # 4 data for R Sharma
sharma4a_main_df = sharma4a_df[['bowling_team', 'over', 'ball', 'bowler']] # Filtered 4 data for Sharma
sharma4a_over_df = sharma4a_main_df[['over']].value_counts() # number of 4's hit per over by Sharma

### R Sharma: 2nd Innings

In [15]:
sharma6b_df = innings2_df.loc[(innings2_df['batsman']=='RG Sharma') &(innings2_df['batsman_runs']==6)] # 6 data for R Sharma
sharma6b_main_df = sharma6b_df[['bowling_team', 'over', 'ball', 'bowler']] # Filtered 6 data for Sharma
sharma6b_over_df = sharma6b_main_df[['over']].value_counts() # number of 6's hit per over by Sharma

In [16]:
sharma4b_df = innings2_df.loc[(innings2_df['batsman']=='RG Sharma') &(innings2_df['batsman_runs']==4)] # 4 data for R Sharma
sharma4b_main_df = sharma4b_df[['bowling_team', 'over', 'ball', 'bowler']] # Filtered 4 data for Sharma
sharma4b_over_df = sharma4b_main_df[['over']].value_counts() # number of 4's hit per over by Sharma

### D Warner: 1st innings

In [17]:
warner6a_df = innings1_df.loc[(innings1_df['batsman']=='DA Warner') &(innings1_df['batsman_runs']==6)] # 6 data for D Warner
warner6a_main_df = warner6a_df[['bowling_team', 'over', 'ball', 'bowler']] # Filtered 6 data for Warner
warner6a_over_df = warner6a_main_df[['over']].value_counts() # number of 6's hit per over by Warner

In [18]:
warner4a_df = innings1_df.loc[(innings1_df['batsman']=='DA Warner') &(innings1_df['batsman_runs']==4)] # 6 data for D Warner
warner4a_main_df = warner4a_df[['bowling_team', 'over', 'ball', 'bowler']] # Filtered 6 data for Warner
warner4a_over_df = warner4a_main_df[['over']].value_counts() # number of 6's hit per over by Warner

### D Warner: 2nd Innings

In [19]:
warner6b_df = innings2_df.loc[(innings2_df['batsman']=='DA Warner') &(innings2_df['batsman_runs']==6)] # 6 data for D Warner
warner6b_main_df = warner6b_df[['bowling_team', 'over', 'ball', 'bowler']] # Filtered 6 data for Warner
warner6b_over_df = warner6b_main_df[['over']].value_counts() # number of 6's hit per over by Warner

In [20]:
warner4b_df = innings2_df.loc[(innings2_df['batsman']=='DA Warner') &(innings2_df['batsman_runs']==4)] # 6 data for D Warner
warner4b_main_df = warner4b_df[['bowling_team', 'over', 'ball', 'bowler']] # Filtered 6 data for Warner
warner4b_over_df = warner4b_main_df[['over']].value_counts() # number of 6's hit per over by Warner

### C Gayle: 1st Innings

In [21]:
gayle6a_df = innings1_df.loc[(innings1_df['batsman']=='CH Gayle') &(innings1_df['batsman_runs']==6)] # 6 data for C Gayle
gayle6a_main_df = gayle6a_df[['bowling_team', 'over', 'ball', 'bowler']] # Filtered 6 data for Gayle
gayle6a_over_df = gayle6a_main_df[['over']].value_counts() # number of 6's hit per over by Gayle

In [22]:
gayle4b_df = innings1_df.loc[(innings1_df['batsman']=='CH Gayle') &(innings1_df['batsman_runs']==4)] # 4 data for C Gayle
gayle4b_main_df = gayle4b_df[['bowling_team', 'over', 'ball', 'bowler']] # Filtered 4 data for Gayle
gayle4b_over_df = gayle4b_main_df[['over']].value_counts() # number of 4's hit per over by Gayle

### C Gayle: 2nd Innings

In [23]:
gayle6b_df = innings2_df.loc[(innings2_df['batsman']=='CH Gayle') &(innings2_df['batsman_runs']==6)] # 6 data for C Gayle
gayle6b_main_df = gayle6b_df[['bowling_team', 'over', 'ball', 'bowler']] # Filtered 6 data for Gayle
gayle6b_over_df = gayle6b_main_df[['over']].value_counts() # number of 6's hit per over by Gayle

In [24]:
# 4's

gayle4b_df = innings2_df.loc[(innings2_df['batsman']=='CH Gayle') &(innings2_df['batsman_runs']==4)] # 4 data for C Gayle
gayle4b_main_df = gayle4b_df[['bowling_team', 'over', 'ball', 'bowler']] # Filtered 4 data for Gayle
gayle4b_over_df = gayle4b_main_df[['over']].value_counts() # number of 4's hit per over by Gayle

### Boundaries Data Visual

### Creating Function for Data Extraction

In [79]:
def get_batsman_info(player, run, innings):
    player_df = deliveries_df[deliveries_df['batsman']==player]
    record_df = player_df.loc[(player_df['inning']==innings)&(player_df['batsman_runs']==run)]
    over_df = record_df[['over']].value_counts().sort_index()
    return over_df

In [83]:
kohli = get_batsman_info('V Kohli', 6, 1)

In [84]:
kohli

over
1        3
2        2
3        4
4        3
5        3
7        2
8        2
9        5
10       7
11       3
12       9
13       6
14       4
15       5
16       5
17      10
18      13
19      10
20      10
dtype: int64

In [116]:
class Player_profile():
    def __init__(self, player):
        self.player_df = deliveries_df[deliveries_df['batsman']==player]
        
    def get_batsman_info(self, run, innings):
        
        self.record_df = self.player_df.loc[(self.player_df['inning']==innings)&(self.player_df['batsman_runs']==run)]
        self.over_df = self.record_df[['over']].value_counts().sort_index()
        return self.over_df

In [117]:
gayle = Player_profile('CH Gayle')

In [118]:
gayle6 = gayle.get_batsman_info(6, 1)

In [120]:
gayle6

over
1        8
2        8
3       12
4       10
5       17
6       12
7        8
8       11
9       14
10       9
11      11
12       8
13      12
14      16
15       9
16       7
17       5
18       4
19       2
20       3
dtype: int64

In [25]:

# df = sharma6b_over_df
 
# # fare v / s class horizontal barplot
# sns.barplot(x = 'over', y = 'total_runs', hue = 'inning', data = df)
 
# # Show the plot
# plt.show()

In [26]:
# sharma_df = deliveries_df.loc[(deliveries_df['batsman']=='RG Sharma') &(deliveries_df['batsman_runs']==6)] # 6 data for R Sharma
# sharma_main_df = sharma_df[['bowling_team', 'over', 'ball', 'bowler']] # Filtered 6 data for Sharma
# sharma_over_df = sharma_main_df[['over']].value_counts() # number of 6's hit per over by Sharma

In [27]:
# gayle6a_over_df = gayle6a_over_df.sort_index()

# # ax = gayle6a_over_df.plot(kind='barh')
# # ax

# sns.barplot(data = gayle6a_over_df, x=gayle6a_over_df.index, y=gayle6a_over_df.values)

# # plt.figure()
# # sns.barplot(gayle6a_over_df.index, gayle6a_over_df.values, alpha=0.8)
# # plt.title('Starbucks in top 10 cities in the World')
# # plt.ylabel('Number of Occurrences', fontsize=12)
# # plt.xlabel('city', fontsize=12)
# # plt.show()

In [28]:
# kohli = kohli6_main_df[['over']].value_counts().sort_index()
# sns.barplot(kohli)

# Data Preparation and Cleaning

# Exploratory Analysis and Visualization

# Questions and Answers

# Infrences and Conclusions

# Refrences and Future Work